In [1]:
import cv2

img_path = "../data/video_frames_4/frame_0000.jpg"
img = cv2.imread(img_path) 

In [2]:
import sys, os
sys.path.append(os.path.abspath('../src'))
from fliqe import FLIQE

fliqe = FLIQE(quality_model_path="../models/encoder_with_binary_head.pth")
FLIQE_score = fliqe.estimate_image_quality(img)
print("FLIQE score:", FLIQE_score)

FLIQE score: 0.9513196349143982


In [3]:
import torch
import pyiqa

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# create metric with default setting
iqa_metric = pyiqa.create_metric('arniqa', device=device)

# check if lower better or higher better
print(iqa_metric.lower_better)

c:\Users\k_chi\vscodeProjects\FLIr Image Quality Estimator (FLIQE)\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False


In [4]:
from pyiqa.archs.niqe_arch import NIQE
from pyiqa.archs.piqe_arch import PIQE
from pyiqa.archs.brisque_arch import BRISQUE
from pyiqa.archs.arniqa_arch import ARNIQA
from pyiqa.archs.paq2piq_arch import PAQ2PIQ
from torchvision import transforms

img_tensor = transforms.ToTensor()(img).unsqueeze(0)

niqe = NIQE()
niqe_score = niqe(img_tensor)
print("NIQE score (lower better):", float(niqe_score))

piqe = PIQE()
piqe_score = piqe(img_tensor)
print("PIQE score (lower better):", float(piqe_score))

brisque = BRISQUE()
brisque_score = brisque(img_tensor)
print("BRISQUE score (lower better):", float(brisque_score.detach()))

arniqa = ARNIQA()
arniqa_score = arniqa(img_tensor)
print("ARNIQA score (higher better):", float(arniqa_score.detach()))

paq2piq = PAQ2PIQ()
paq2piq.eval()  # Ensure model is in evaluation mode
paq2piq_score = paq2piq(img_tensor)
print("PAQ2PIQ score (higher better):", float(paq2piq_score.detach()))

NIQE score (lower better): 3.859058813628418
PIQE score (lower better): 29.576688766479492
BRISQUE score (lower better): 10.10565185546875
ARNIQA score (higher better): 0.5652608871459961
Loading pretrained model PAQ2PIQ from C:\Users\k_chi\.cache\torch\hub\pyiqa\P2P_RoIPoolModel-fit.10.bs.120-ca69882e.pth
PAQ2PIQ score (higher better): 68.7802963256836


In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import torch
import torch.nn as nn
from tqdm import tqdm
import sys, os
sys.path.append(os.path.abspath('../src'))
from fliqe.datasets import ImageDataset
from fliqe.distortions import *
from torchvision import transforms
from torch.utils.data import DataLoader


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(f"Using device: {device}")
    
distortions = [Clean(), LensBlur(), MotionBlur(), GaussianNoise(), Overexposure(), Underexposure(), Compression(), Ghosting(), Aliasing()]
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)), # 224 or 384
    transforms.ToTensor(),
])

# All folders with images
all_folders = [f"../data/video_frames_{i}" for i in range(1, 13)]
# Gather all image paths
all_image_paths = []
for folder in all_folders:
    if not os.path.isdir(folder):
        print(f"Warning: folder {folder} not found, skipping.")
        continue
    all_image_paths.extend(
        [os.path.join(folder, fname)
         for fname in os.listdir(folder)
         if fname.lower().endswith(('.jpg', '.png'))]
    )
print(f"Total images found: {len(all_image_paths)}")

# Shuffle and split
random.shuffle(all_image_paths)
split_ratio = 0.8  # 80% train, 20% eval
split_idx = int(len(all_image_paths) * split_ratio)
train_paths = all_image_paths[:split_idx]
eval_paths = all_image_paths[split_idx:]

# Create datasets
train_dataset = ImageDataset(train_paths, distortions=distortions, transform=transform, binary_labels=True)
eval_dataset = ImageDataset(eval_paths, distortions=distortions, transform=transform, binary_labels=True)

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True,
                              num_workers=16, pin_memory=True, persistent_workers=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=1, shuffle=False,
                             num_workers=16, pin_memory=True, persistent_workers=True)

print(f"Train Dataset length: {len(train_dataset)}")
print(f"Eval Dataset length: {len(eval_dataset)}")

train_scores, train_labels = [], []
eval_scores, eval_labels = [], []

def compute_scores(dataloader, score_list, label_list):
    for imgs, labels in tqdm(dataloader, desc="Computing Scores", leave=False):
        imgs = imgs.to(device)
        binary_labels = torch.tensor(
            [0.0 if l == 'Clean' else 1.0 for l in labels],
            dtype=torch.float32, device=device
        ).unsqueeze(1)
        # NIQE expects image tensors in a specific format — adjust if needed
        scores = niqe(imgs)
        score_list.extend(scores)
        label_list.extend(binary_labels.cpu().numpy().flatten())


# Compute for training set
compute_scores(train_dataloader, train_scores, train_labels)

# Compute for evaluation set
compute_scores(eval_dataloader, eval_scores, eval_labels)

print(f"Train: {len(train_scores)} scores, Eval: {len(eval_scores)} scores")

2.8.0+cu129
True
12.9
91002
Using device: cuda
Total images found: 80670
Train Dataset length: 64536
Eval Dataset length: 16134


TypeError: iteration over a 0-d tensor

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report

# convert lists to arrays
X_train = np.array(train_scores).reshape(-1, 1)
y_train = np.array(train_labels)

X_eval = np.array(eval_scores).reshape(-1, 1)
y_eval = np.array(eval_labels)

# Fit logistic regression (Platt scaling)
platt = LogisticRegression(solver="lbfgs", max_iter=1000)
platt.fit(X_train, y_train)

# Calibrated probabilities
train_probs = platt.predict_proba(X_train)[:, 1]
eval_probs  = platt.predict_proba(X_eval)[:, 1]

# Default threshold 0.5
eval_preds_05 = (eval_probs >= 0.5).astype(int)
print("Eval F1 (threshold 0.5):", f1_score(y_eval, eval_preds_05))
print(classification_report(y_eval, eval_preds_05))